In [ ]:
# Get a comp demo data
comp_demo = db.get_table(library='comp', table='funda', columns=['gvkey', 'datadate',
                                                                'fyear'])
comp_demo.drop_duplicates(inplace=True)

# Get a CRSP demo data
crsp_demo = db.get_table(library='crsp', table='msf', columns=['permno', 'ncusip', 'date'])
crsp_demo.drop_duplicates(inplace=True)

# Get Compustat-CRSP linktable
link = db.get_table(library='crsp', table='ccmxpf_lnkhist', columns=['gvkey', 'lpermno', 'lpermco', 'linktype', 'linkprim']) # 'linkdt', 'linkenddt'
link = link[link['linktype'].isin(['LC', 'LU'])]
link = link[link['linkprim'].isin(['C', 'P'])]
link.drop(['linktype', 'linkprim'], axis=1, inplace=True)

crsp_add_gvkey = crsp_demo.merge(link, left_on='permno', right_on='lpermno')
crsp_add_gvkey.drop_duplicates(inplace=True)

out = crsp_add_gvkey.merge(comp_demo, left_on='gvkey', right_on='gvkey')
out.drop_duplicates(inplace=True)

```python
import wrds
db = wrds.Connection(wrds_username='kpu2019')
comp_funda_describe = db.describe_table('comp', 'funda')
crsp_msf_describe = db.describe_table('crsp', 'msf')
```

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
# import tushare as ts
import wrds
import talib as ta
import datetime, time

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [4]:
db = wrds.Connection(wrds_username='kpu2019')

Loading library list...
Done


In [8]:
db.describe_table('crsp', 'ccmxpf_linktable')

Approximately 76486 rows in crsp.ccmxpf_linktable.


,name,nullable,type
0,gvkey,True,VARCHAR(6)
1,linkprim,True,VARCHAR(1)
2,liid,True,VARCHAR(3)
3,linktype,True,VARCHAR(2)
4,lpermno,True,DOUBLE PRECISION
5,lpermco,True,DOUBLE PRECISION
6,usedflag,True,DOUBLE PRECISION
7,linkdt,True,DATE
8,linkenddt,True,DATE


In [9]:
db.describe_table('crsp', 'ccmxpf_lnkhist')

Approximately 101711 rows in crsp.ccmxpf_lnkhist.


,name,nullable,type
0,gvkey,True,VARCHAR(6)
1,linkprim,True,VARCHAR(1)
2,liid,True,VARCHAR(3)
3,linktype,True,VARCHAR(2)
4,lpermno,True,DOUBLE PRECISION
5,lpermco,True,DOUBLE PRECISION
6,linkdt,True,DATE
7,linkenddt,True,DATE


In [10]:
ccm = db.get_table('crsp', 'ccmxpf_linktable', coerce_float=False, 
                   date_cols=['linkdt', 'linkenddt'])

In [13]:
ccm.head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,usedflag,linkdt,linkenddt
0,001000,C,00X,NU,NaN,NaN,-1.0,1961-01-01,1970-09-29
1,001000,P,01,NU,NaN,NaN,-1.0,1970-09-30,1970-11-12
2,001000,P,01,LU,25881.0,23369.0,1.0,1970-11-13,1978-06-30
3,001001,C,00X,NU,NaN,NaN,-1.0,1978-01-01,1983-09-19
4,001001,P,01,LU,10015.0,6398.0,1.0,1983-09-20,1986-07-31


In [18]:
data_crsp = pd.read_csv('data/return.zip', 
                        parse_dates=['date'], 
                        infer_datetime_format=True, )
#                         dtype={'PERMCO':str, 'CUSIP':str})

E:\ProgramData\Anaconda3\envs\quant\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
1.0 == 1

True

In [19]:
data_crsp.CUSIP.value_counts()

48783610    432
16389320    432
78463510    432
71404610    432
06050510    432
37733W10    432
45950610    432
81707010    432
94262210    432
95058W10    432
91382130    432
89543610    432
19416210    432
89417E10    432
66641610    432
62671710    432
29101110    432
98939010    432
73912810    432
37033410    432
87889530    432
56633010    432
29364G10    432
26000310    432
01880210    432
52605710    432
05361110    432
65334H10    432
74586710    432
20038K10    432
           ... 
25490K81      1
67984210      1
68386C60      1
19650410      1
92211910      1
29642510      1
78464A33      1
92189F83      1
60156610      1
23323710      1
89099090      1
55171510      1
37322610      1
923010        1
97239110      1
92828Q10      1
90253910      1
04280210      1
37038810      1
83172F20      1
13680710      1
60872720      1
32529710      1
85356410      1
25720110      1
29383920      1
66745610      1
18383Q81      1
97209910      1
58133410      1
Name: CUSIP, Length: 265